In [1]:
%run functions.ipynb
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
X_train, X_test, y_train, y_test = load_train_and_test_parts()
print("Train and test sizes: {} {}".format(X_train.shape, X_test.shape))
print("(1, 0) labels count in train test: {} {}".format((np.count_nonzero(y_train==1), np.count_nonzero(y_train==0)), 
                                                        (np.count_nonzero(y_test==1), np.count_nonzero(y_test==0))))

Train and test sizes: (726, 12179) (243, 12179)
(1, 0) labels count in train test: (289, 437) (104, 139)


In [3]:
scoring = 'roc_auc'
cv = 4
cv_out = 5
cv_in = 2
n_iter = 1
n_jobs = 3

In [4]:
print('n_iter: {}\ncv: {}\nn_jobs: {}\nscoring: {}'.format(n_iter, cv, n_jobs, scoring))

n_iter: 3
cv: 4
n_jobs: 3
scoring: roc_auc


In [5]:
nsc_params = {'shrink_threshold' : np.linspace(0, 5, 50)}
clf_nsc = RandomizedSearchCV(NearestCentroidWrapper(), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, iid=False, 
                             param_distributions=nsc_params, random_state=random_state)  
clf_nsc = fit_clf_print_scores(clf_nsc, X_train, y_train, X_test, y_test)

{'shrink_threshold': 3.979591836734694}


,ROC AUC,Precision,Recall,F1
train,0.682,0.601,0.647,0.623
test,0.634,0.583,0.577,0.580


In [6]:
clf_lda = fit_clf_print_scores(LinearDiscriminantAnalysis(shrinkage='auto', solver='lsqr'), X_train, y_train, X_test, y_test)

,ROC AUC,Precision,Recall,F1
train,1.000,1.000,1.000,1.000
test,0.796,0.753,0.615,0.677


In [7]:
logistic_params = {'C' : np.linspace(0.01, 0.6, 50), 'penalty' : ['l1'], 'solver' : ['liblinear']}
tree_params = {'n_estimators' : [1000, 2500, 5000], 'max_depth': [4, 6, 8], 'min_samples_leaf' : [20, 30], 
               "bootstrap": [True, False], "criterion": ["gini", "entropy"]}

In [8]:
logit_cv = RandomizedSearchCV(LogisticRegression(random_state=random_state), cv=cv, scoring=scoring, random_state=random_state,
                              n_iter=n_iter, n_jobs=n_jobs, iid=False, param_distributions=logistic_params) 
tree_cv = RandomizedSearchCV(RandomForestClassifier(random_state=random_state), cv=cv, scoring=scoring, random_state=random_state,
                             n_iter=n_iter, n_jobs=n_jobs, iid=False, param_distributions=tree_params) 

clf_logit = fit_clf_print_scores(logit_cv, X_train, y_train, X_test, y_test)
clf_forest = fit_clf_print_scores(tree_cv, X_train, y_train, X_test, y_test)

{'solver': 'liblinear', 'penalty': 'l1', 'C': 0.16653061224489796}


,ROC AUC,Precision,Recall,F1
train,0.957,0.911,0.848,0.878
test,0.829,0.724,0.683,0.703


{'n_estimators': 5000, 'min_samples_leaf': 20, 'max_depth': 6, 'criterion': 'entropy', 'bootstrap': False}


,ROC AUC,Precision,Recall,F1
train,0.993,0.992,0.872,0.928
test,0.788,0.760,0.548,0.637


In [ ]:
elastic_params = {'l1_ratio' : np.linspace(0.001, 1, 50), 'alpha' : np.linspace(0.0001, 0.5, 100)}
elastic_cv = RandomizedSearchCV(SGDClassifier(loss='log', penalty='elasticnet', max_iter=5000, tol=1e-3, random_state=random_state), 
                                cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, random_state=random_state,
                                iid=False, param_distributions=elastic_params)
clf_en = fit_clf_print_scores(elastic_cv, X_train, y_train, X_test, y_test)

{'l1_ratio': 0.20487755102040817, 'alpha': 0.08594141414141414}


,ROC AUC,Precision,Recall,F1
train,0.861,0.700,0.775,0.736
test,0.804,0.613,0.731,0.667


In [ ]:
svm_params = {'C' : np.linspace(0.01, 0.6, 50)}
svm_cv = RandomizedSearchCV(LinearSVC(penalty='l1', dual=False, max_iter=10000, random_state=random_state), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, 
                                iid=False, param_distributions=svm_params, random_state=random_state)

clf_svm = fit_clf_print_scores(svm_cv, X_train, y_train, X_test, y_test)

{'C': 0.3712244897959183}


/pio/scratch/1/i272955/anaconda3/envs/master/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


,ROC AUC,Precision,Recall,F1
train,1.000,1.000,1.000,1.000
test,0.782,0.704,0.663,0.683


In [ ]:
ada_params = {'n_estimators' : [100, 250, 500], 
              "learning_rate": [0.01, 0.1, 1, 10],
              "base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "base_estimator__max_depth": [1, 2, 4, 8, None], 
              "base_estimator__min_samples_split": [25, 50, 100] 
             }
ada_cv = RandomizedSearchCV(AdaBoostClassifier(base_estimator= DecisionTreeClassifier(min_samples_leaf=10), random_state=random_state), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, 
                                iid=False, param_distributions=ada_params, random_state=random_state)

clf_ada = fit_clf_print_scores(ada_cv, X_train, y_train, X_test, y_test)

{'n_estimators': 100, 'learning_rate': 1, 'base_estimator__splitter': 'best', 'base_estimator__min_samples_split': 100, 'base_estimator__max_depth': 8, 'base_estimator__criterion': 'gini'}


,ROC AUC,Precision,Recall,F1
train,1.000,1.000,1.000,1.000
test,0.773,0.681,0.596,0.636


In [ ]:
rlr_params = {'C' : np.linspace(0.1, 5, 50), 'n_estimators' : [100, 250, 500, 1000], 'n_variables' : [100, 250, 500, 1000]}
rlr_cv = RandomizedSearchCV(RandomLogisticsRegressions(penalty='l1'), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, 
                                iid=False, param_distributions=rlr_params, random_state=random_state)

clf_rlr = fit_clf_print_scores(rlr_cv, X_train, y_train, X_test, y_test)